In [1]:
import GetPropertiesAPI as GP
from collections import Counter
import pandas as pd
import importlib
importlib.reload(GP)
import statistics as s
import operator
from collections import defaultdict
import csv 
import matplotlib.pyplot as plt
import numpy as np

In [107]:
aid_list = []
gidAidDict = {}
for gid in range(1,9407):
    aid = GP.getAnnotID(gid)
    aid_list.append(aid)
    gidAidDict[gid] = aid

aid_list = list(filter(lambda x: x != None,aid_list))

{1: [13386], 2: [13416], 3: [13425], 4: [13399], 5: [13423], 6: [13385], 7: [13453], 8: [13390], 9: [9332], 10: [9336], 11: [9329], 12: [9330], 13: [9340], 14: [9331], 15: [9325], 16: [9326], 17: [9327], 18: [9334], 19: [9335], 20: [9321], 21: [9328], 22: [9338], 23: [9323], 24: None, 25: None, 26: [13458], 27: [13417], 28: [13424], 29: [13387], 30: [13401, 13402], 31: [13392], 32: [13426], 33: [13436], 34: [13429], 35: [13435], 36: [13441], 37: [13405], 38: [13442], 39: [13451], 40: [9337], 41: [9339], 42: [9333], 43: [13456], 44: [13413], 45: [13412], 46: [13447], 47: [13394, 13395], 48: [13444], 49: [13407, 13408], 50: [13427], 51: [13438], 52: [13437], 53: [13411], 54: [13415], 55: [13404], 56: [13389], 57: [13391], 58: [13445], 59: [13422], 60: [13446], 61: [13428], 62: [13432], 63: [13440], 64: [13409, 13410], 65: [13414], 66: [13418], 67: [13455], 68: [13457], 69: [13403], 70: [13406], 71: [13398], 72: [13431], 73: [13419, 13420], 74: [13448], 75: [13400], 76: [13396], 77: [1343

In [109]:
print(len(gidAidDict.keys()))

9406


In [16]:
aids = []
for aid_l in aid_list:
    for aid in aid_l:
        aids.append(aid)

In [20]:
aidContribTupList = []
for aid in aids:
    contrib = GP.getImageFeature(aid,'image_contributor_tag')
    aidContribTupList.append((aid,contrib[0]))

In [21]:
aidNidMap = {}
aidNamesMap = {}
aidNidTupList = [] # modified

for aid in aids:
    nid = GP.getImageFeature(aid,'nids')
    aidNidMap[aid] = nid
    aidNidTupList.append((aid,nid[0]))

In [22]:
nids = []
for aid in aidNidMap.keys():
    nids.append(aidNidMap[aid][0])

In [23]:
nids = list(filter(lambda x : x > 0,nids))

In [9]:
counter_nid= Counter(nids)


In [10]:
gidAidDict
aidGidTupList = [] # key : aid and value : gid # modified
for gid in gidAidDict.keys():
    if gidAidDict[gid] != None:
        for aid in gidAidDict[gid]:
            aidGidTupList.append((aid,gid))

In [11]:
aidGidDf = pd.DataFrame(aidGidTupList,columns = ['AID','GID']) 
aidNidDf = pd.DataFrame(aidNidTupList,columns = ['AID','NID']) 
aidContribDf = pd.DataFrame(aidContribTupList,columns = ['AID','CONTRIBUTOR'])
aidNidDf = aidNidDf[(aidNidDf['NID']>0)]

In [12]:
aidGidNidDf = pd.merge(aidGidDf,aidNidDf,left_on = 'AID',right_on = 'AID')
aidGidNidContribDf = pd.merge(aidGidNidDf,aidContribDf,left_on = 'AID',right_on = 'AID')

In [13]:
aidGidNidContribDf.to_csv('results.csv',index=False)

In [157]:
with open('results.csv') as f: # read from csv file into a key : GID and value : CONTRIBUTOR
    reader = csv.DictReader(f)
    gidContribMap = { line['GID']: line['CONTRIBUTOR'] for line in reader }
        
len(gidContribMap)

4948

In [277]:
ContribTotal = {} # dict with key : CONTRIBUTOR and value: Total photos taken

for gid,contrib in gidContribMap.items():
     ContribTotal[contrib] = ContribTotal.get(contrib,0) + 1

print(s.mean(ContribTotal.values()))
print(s.stdev(ContribTotal.values()))

88.35714285714286
136.7916037523607


In [2]:
with open('results.csv') as f2: # read from csv file into a Dict with key : AID and value : GID, NID, CONTRIBUTOR
    reader2 = csv.DictReader(f2)
    aidToGidNidContribMap = { line['AID']: [line['GID'], line['NID'], line['CONTRIBUTOR']] for line in reader2 }

In [114]:
# needed for unknowns 9406 - 4948
nidAidMap =defaultdict(list)
for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
     nidAidMap[gid].append(aid)
        
len(nidAidMap.keys())

4948

In [275]:
NidContribTotal = {} # dict with key : NID, CONTRIBUTOR and value: Total photos taken
for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
     NidContribTotal[gid] = NidContribTotal.get(nid)

In [234]:
csv_out = csv.writer(open('nidtoContributor.csv', 'w')) # rename file, results of Nid to Contributor to Total
csv_out.writerow(['NID', 'CONTRIBUTOR', 'TOTAL'])
for (Nid, Contrib), value in NidContribTotal.items():
   csv_out.writerow([Nid, Contrib, value])

In [272]:
#from collections import defaultdict

averageCountofPictures = defaultdict(list) # dict where key : NID and values: list of pictures taken per photographer

for (nid, contrib), total in NidContribTotal.items(): 
    averageCountofPictures[nid].append(total)
       
#averageCountofPictures

In [276]:
countUniquePhotoPerPic = {} # dict where key : NID and values : # of CONTRIBUTERS

for (nid, contrib), total in NidContribTotal.items():
    countUniquePhotoPerPic[nid] = countUniquePhotoPerPic.get((nid),0) + 1 

#countUniquePhotoPerPic['741']

In [ ]:
#JUST LOOK FROM HERE, TOP PART JOSE IS TAKING CARE OF ################################

In [11]:
# Arguments : CSV file, Required Feature
# Accepted Features: species_texts, age_months_est, exemplar_flags, sex_texts, yaw_texts, quality_texts,image_contributor_tag
# Returns : None
# Creates: specific feature csv file, all feature csv file, all feature csv file with sum and plot
def getContributorFeature(filename, feature):
    
    with open(filename) as f: # read from csv file into a Dict with key : AID and value : GID, NID, CONTRIBUTOR
        reader = csv.DictReader(f)
        aidToGidNidContribMap = { line['AID']: [line['GID'], line['NID'], line['CONTRIBUTOR']] for line in reader }
    
    contribToFeatureMap = defaultdict(list) # dict where key : contributor and values : List of feature
    for aid,(gid,nid,contrib) in aidToGidNidContribMap.items():
        if feature != "age/months":
            contribToFeatureMap[contrib].append(GP.getImageFeature(aid, feature)[0])
        else:
            contribToFeatureMap[contrib].append(GP.getImageFeature(aid, feature))
    
    if feature == "age/months":
        contribToFeatureMap = dict(contribToFeatureMap)
        result = {}

        for contrib in contribToFeatureMap.keys():
             result[contrib] = [GP.getAgeFeatureReadableFmt(i) for i in contribToFeatureMap[contrib]]
        
        result2=defaultdict(list)
        for contrib, age_list in result.items():
            for age in age_list:
                for ind_age in age:
                    result2[contrib].append(ind_age)
        
        contribAnimFeatCount = {} # dict where key : contributor and values : total of specific feature
        for key in result2.keys():
            contribAnimFeatCount[key]=dict(Counter(result2[key]))
            
        getContributorSpecificFeature(contribAnimFeatCount, 'juveniles- two year old')
        getContributorSpecificFeature(contribAnimFeatCount, 'juveniles - one year old')
        getContributorSpecificFeature(contribAnimFeatCount, 'infant')
        getContributorSpecificFeature(contribAnimFeatCount, 'adult')
        getContributorSpecificFeature(contribAnimFeatCount, 'unknown')
        
        createcombinedcsv("age/months", "juveniles2yearold", 'juveniles1yearold', 'infant', 'adult', 'unknown')
        
        sumRows('juveniles2yearold-unknown.csv')
        
        createStackgraph("sumjuveniles2yearold-unknown.csv", "age/months", "juveniles2yearold", 'juveniles1yearold', 'infant', 'adult', 'unknown')
        
        return # no need to continue after this

    
    # put this code here because has to do something different than the other ones
    if feature == "exemplar":
        exemplarDict=defaultdict(list)  #change 0 and 1 to 'zero' and 'one' so getContributorSpecificFeature method can work
        for contrib, value in contribToFeatureMap.items():  
            for ind in value:
                if ind == 0:
                    ind ='zero'
                else:
                    ind ='one'
                exemplarDict[contrib].append(ind)
            
        exemplarRegDict={}
        for key in temp.keys():
            exemplarRegDict[key]=dict(Counter(exemplarDict[key]))     
        
        getContributorSpecificFeature(exemplarRegDict, 'zero')
        getContributorSpecificFeature(exemplarRegDict, 'one')
            
        createcombinedcsv("exemplar","zero", "one")
        
        sumRows('zeroandone.csv')
        
        createStackgraph("sumzeroandone.csv", "exemplar", "zero", "one")
        
        return # no need to continue after this
    
    #NOT "exemplar_flags" or 'age_months_est' so continue on
    
    contribAnimFeatCount = {} # dict where key : contributor and values : total of specific feature
    for key in contribToFeatureMap.keys():
        contribAnimFeatCount[key]=dict(Counter(contribToFeatureMap[key]))

    if feature == "sex/text":  
        # get each individual csv files where Contributor, SpecFeature
        getContributorSpecificFeature(contribAnimFeatCount, 'Male')
        getContributorSpecificFeature(contribAnimFeatCount, 'Female')
        getContributorSpecificFeature(contribAnimFeatCount, "UNKNOWN SEX")
        
        # combine previous csv files
        createcombinedcsv("sex/text", "Male", "Female", "UnknownSex")
        
        # include sum column in csv file
        sumRows('Male-UnknownSex.csv')
        
        # plots
        createStackgraph("sumMale-UnknownSex.csv", "sex/text", "Male", "Female", "UnknownSex")
        
    elif feature == "species/text":
        
        getContributorSpecificFeature(contribAnimFeatCount, "giraffe_masai")
        getContributorSpecificFeature(contribAnimFeatCount, "zebra_plains")
        
        createcombinedcsv("species/text","giraffe_masai", "zebra_plains")
        
        sumRows('giraffe_masaiandzebra_plains.csv')
        
        createStackgraph("sumgiraffe_masaiandzebra_plains.csv", "species/text", "giraffe_masai", "zebra_plains")
        
    elif feature == "yaw/text":
        getContributorSpecificFeature(contribAnimFeatCount, "front")
        getContributorSpecificFeature(contribAnimFeatCount, "back")
        getContributorSpecificFeature(contribAnimFeatCount, "left")
        getContributorSpecificFeature(contribAnimFeatCount, "right")
        getContributorSpecificFeature(contribAnimFeatCount, "frontleft")
        getContributorSpecificFeature(contribAnimFeatCount, "frontright")
        getContributorSpecificFeature(contribAnimFeatCount, "backleft")
        getContributorSpecificFeature(contribAnimFeatCount, "backright")
        
        createcombinedcsv("yaw/text", "front", "back", "left", "right", "frontleft", "frontright", "backleft", "backright")
        
        sumRows('yaw_texts.csv')
        
        createStackgraph("sumyaw_texts.csv", "yaw/text", "front", "back", "left", "right", "frontleft", "frontright", "backleft", "backright")
        
    elif feature == "quality/text":
        getContributorSpecificFeature(contribAnimFeatCount, "excellent")
        getContributorSpecificFeature(contribAnimFeatCount, "good")
        getContributorSpecificFeature(contribAnimFeatCount, "ok")
        getContributorSpecificFeature(contribAnimFeatCount, "poor")
        getContributorSpecificFeature(contribAnimFeatCount, "junk")
        
        createcombinedcsv("quality/text", "excellent", "good", "ok", "poor", "junk")
        
        sumRows('excellent-junk.csv')
        
        createStackgraph("sumexcellent-junk.csv", "quality/text", "excellent", "good", "ok", "poor", "junk")
    
    # have to do something new for image contributor tag   
    elif feature == 'image/contributor/tag':    #ONLY NUMBERS SO NO NEED TO CALL HELPER FUNCTION, NOT SORTED
        
        csv_out = csv.writer(open('contribImage_Contributor_TagMap.csv', 'w')) 
        csv_out.writerow(['CONTRIBUTOR', "image_contributor_tag"])
        for contrib, contrib2 in contribAnimFeatCount.items(): #DICTIONARY WITHIN A DICTIONARY
            for contrib3, total in contrib2.items():
                csv_out.writerow([contrib3, total])
                
        creategraph('contribImage_Contributor_TagMap.csv', feature)
    
    else:
        print("WRong feature, SOMETHING IS WRONG")

In [12]:
#getContributorFeature(aidToGidNidContribMap, "sex/text")
#getContributorFeature(aidToGidNidContribMap, 'age/months')
#getContributorFeature(aidToGidNidContribMap, "exemplar") 
#getContributorFeature(aidToGidNidContribMap, "species/text") 
#getContributorFeature(aidToGidNidContribMap, "yaw/text") 
#getContributorFeature(aidToGidNidContribMap, "quality/text") 
x = getContributorFeature('results.csv', "image/contributor/tag")
x

{'GIRM_MUGU_20,hyrule:joncrall:/media/raid/work/GIRM_MUGU_20,,,,,0': {'GIRM_MUGU_20,hyrule:joncrall:/media/raid/work/GIRM_MUGU_20,,,,,0': 38},
 "NNP GZC Car '10WHITE', Person 'A'": {"NNP GZC Car '10WHITE', Person 'A'": 16},
 "NNP GZC Car '11WHITE', Person 'A'": {"NNP GZC Car '11WHITE', Person 'A'": 139},
 "NNP GZC Car '12WHITE', Person 'A'": {"NNP GZC Car '12WHITE', Person 'A'": 31},
 "NNP GZC Car '13WHITE', Person 'A'": {"NNP GZC Car '13WHITE', Person 'A'": 23},
 "NNP GZC Car '13WHITE', Person 'B'": {"NNP GZC Car '13WHITE', Person 'B'": 65},
 "NNP GZC Car '14WHITE', Person 'A'": {"NNP GZC Car '14WHITE', Person 'A'": 110},
 "NNP GZC Car '15WHITE', Person 'A'": {"NNP GZC Car '15WHITE', Person 'A'": 71},
 "NNP GZC Car '15WHITE', Person 'B'": {"NNP GZC Car '15WHITE', Person 'B'": 9},
 "NNP GZC Car '15WHITE', Person 'C'": {"NNP GZC Car '15WHITE', Person 'C'": 6},
 "NNP GZC Car '15WHITE', Person 'D'": {"NNP GZC Car '15WHITE', Person 'D'": 30},
 "NNP GZC Car '16WHITE', Person 'A'": {"NNP GZC

In [17]:
for contrib, contrib2 in x.items():
    for contrib3, total in contrib2.items():
        print([contrib3, total])

["NNP GZC Car '1PURPLE', Person 'D'", 515]
["NNP GZC Car '3RED', Person 'A'", 78]
["NNP GZC Car '10WHITE', Person 'A'", 16]
['PZ_MUGU_18,hyrule:joncrall:/media/raid/work/PZ_MUGU_18,,,,,0', 368]
["NNP GZC Car '2RED', Person 'A'", 127]
["NNP GZC Car '3PURPLE', Person 'B'", 43]
['NNP_Master,pachy.cs.uic.edu:jonc:/home/shared_ibeis/data/work/NNP_Master,,,,,6', 1309]
["NNP GZC Car '1WHITE', Person 'A'", 105]
["NNP GZC Car '1BLUE', Person 'D'", 14]
["NNP GZC Car '17WHITE', Person 'C'", 15]
["NNP GZC Car '15WHITE', Person 'B'", 9]
["NNP GZC Car '1RED', Person 'B'", 268]
["NNP GZC Car '1PURPLE', Person 'B'", 430]
["NNP GZC Car '2RED', Person 'C'", 118]
["NNP GZC Car '9WHITE', Person 'A'", 27]
["NNP GZC Car '5WHITE', Person 'A'", 151]
['GIRM_MUGU_20,hyrule:joncrall:/media/raid/work/GIRM_MUGU_20,,,,,0', 38]
["NNP GZC Car '2RED', Person 'D'", 57]
["NNP GZC Car '3PURPLE', Person 'A'", 11]
["NNP GZC Car '4WHITE', Person 'A'", 23]
["NNP GZC Car '1RED', Person 'A'", 115]
["NNP GZC Car '6WHITE', Perso

In [2]:
# Arguments : ContributorToFeatureDict , Required Specific Feature
# Accepted Specific Features: 
    # sex/text = "Male", "Female", "UNKNOWN SEX".
    # species/text = "giraffe_masai", "zebra_plains"
    # exemplar = "zero", "one"
    # yaw/text = "front", "back", "left", "right", "frontleft", "frontright", "backleft", "backright"
    # quality/text = "excellent", "good", "ok", "poor", "junk" 
# Creates: csv file 
def getContributorSpecificFeature(contribAnimFeatCount, specificfeat):
    contribSpecFeatureMap={}
    
    for contrib, feature in contribAnimFeatCount.items():
        contribSpecFeatureMap[contrib]=feature.get(specificfeat , 0)
        
    
    sortedcontribSpecFeatureMap=sorted(contribSpecFeatureMap.items(),key = operator.itemgetter(0))
    
    # for later use because UNKNOWN SEX is two words and must be only one
    if (specificfeat == "UNKNOWN SEX"):
        specificfeat = 'UnknownSex'
    elif (specificfeat == 'juveniles- two year old'):
        specificfeat = 'juveniles2yearold'
    elif  (specificfeat=="juveniles - one year old"):
        specificfeat = 'juveniles1yearold'
    
    csv_out = csv.writer(open('contrib'+ specificfeat +'Map.csv', 'w')) 
    csv_out.writerow(['CONTRIBUTOR', specificfeat])
    for row in sortedcontribSpecFeatureMap:
        csv_out.writerow(row)

In [ ]:
def getContirb

In [3]:
# Arguments : Feature name first the specific features in a list
# Returns : None
# Creates: csv file
def createcombinedcsv(*args):
    with open('contrib' + args[1] + 'Map.csv') as f:
        next(f)
        data=[tuple(line) for line in csv.reader(f)]
    
    with open('contrib' + args[2] + 'Map.csv') as f:
        next(f)
        data1=[tuple(line) for line in csv.reader(f)]
        
    if args[0] == "species/text" or args[0] == "exemplar":
        contribFeat1Df = pd.DataFrame(data,columns = ['CONTRIBUTOR',args[1]]) 
        contribFeat2Df = pd.DataFrame(data1,columns = ['CONTRIBUTOR',args[2]]) 
    
        Feat1Feat2Merge = pd.merge(contribFeat1Df,contribFeat2Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
    
        Feat1Feat2Merge.to_csv(args[1] + 'and' + args[2] + '.csv',index=False)
    elif args[0] == "sex/text":
        with open('contrib' + args[3] + 'Map.csv') as f:
            next(f)
            data2=[tuple(line) for line in csv.reader(f)]
    
        contribFeat1Df = pd.DataFrame(data,columns = ['CONTRIBUTOR', args[1]]) 
        contribFeat2Df = pd.DataFrame(data1,columns = ['CONTRIBUTOR', args[2]]) 
        contribFeat3Df = pd.DataFrame(data2,columns = ['CONTRIBUTOR', args[3]])

        Feat1Feat2Merge = pd.merge(contribFeat1Df,contribFeat2Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        FinalContribDf = pd.merge(Feat1Feat2Merge,contribFeat3Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')

        FinalContribDf.to_csv(args[1]+ '-' + args[3] + '.csv',index=False)
        
    elif args[0] == "quality/text" or args[0] == "age/months":
        with open('contrib' + args[3] + 'Map.csv') as f:
            next(f)
            data2=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[4] + 'Map.csv') as f:
            next(f)
            data3=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[5] + 'Map.csv') as f:
            next(f)
            data4=[tuple(line) for line in csv.reader(f)]
     
        contribFeat1Df = pd.DataFrame(data,columns = ['CONTRIBUTOR', args[1]]) 
        contribFeat2Df = pd.DataFrame(data1,columns = ['CONTRIBUTOR', args[2]]) 
        contribFeat3Df = pd.DataFrame(data2,columns = ['CONTRIBUTOR',args[3]])
        contribFeat4Df = pd.DataFrame(data3,columns = ['CONTRIBUTOR',args[4]])
        contribFeat5Df = pd.DataFrame(data4,columns = ['CONTRIBUTOR',args[5]])
    
        Feat1Feat2Merge = pd.merge(contribFeat1Df,contribFeat2Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat2Feat3Merge = pd.merge(Feat1Feat2Merge,contribFeat3Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat3Feat4Merge = pd.merge(Feat2Feat3Merge,contribFeat4Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        FinalContribDf = pd.merge(Feat3Feat4Merge,contribFeat5Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
    
        FinalContribDf.to_csv(args[1] + "-" + args[5] + '.csv',index=False)
        
    elif args[0] == "yaw/text":
        with open('contrib' + args[3] + 'Map.csv') as f:
            next(f)
            data2=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[4] + 'Map.csv') as f:
            next(f)
            data3=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[5] + 'Map.csv') as f:
            next(f)
            data4=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[6] + 'Map.csv') as f:
            next(f)
            data5=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[7] + 'Map.csv') as f:
            next(f)
            data6=[tuple(line) for line in csv.reader(f)]
        
        with open('contrib' + args[8] + 'Map.csv') as f:
            next(f)
            data7=[tuple(line) for line in csv.reader(f)]
        
        contribFeat1Df = pd.DataFrame(data,columns = ['CONTRIBUTOR', args[1]]) 
        contribFeat2Df = pd.DataFrame(data1,columns = ['CONTRIBUTOR', args[2]]) 
        contribFeat3Df = pd.DataFrame(data2,columns = ['CONTRIBUTOR',args[3]])
        contribFeat4Df = pd.DataFrame(data3,columns = ['CONTRIBUTOR',args[4]])
        contribFeat5Df = pd.DataFrame(data4,columns = ['CONTRIBUTOR',args[5]])
        contribFeat6Df = pd.DataFrame(data5,columns = ['CONTRIBUTOR',args[6]])
        contribFeat7Df = pd.DataFrame(data6,columns = ['CONTRIBUTOR',args[7]])
        contribFeat8Df = pd.DataFrame(data7,columns = ['CONTRIBUTOR',args[8]])

        Feat1Feat2Merge = pd.merge(contribFeat1Df,contribFeat2Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat2Feat3Merge = pd.merge(Feat1Feat2Merge,contribFeat3Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat3Feat4Merge = pd.merge(Feat2Feat3Merge,contribFeat4Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat4Feat5Merge = pd.merge(Feat3Feat4Merge,contribFeat5Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat5Feat6Merge = pd.merge(Feat4Feat5Merge,contribFeat6Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        Feat6Feat7Merge = pd.merge(Feat5Feat6Merge,contribFeat7Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')
        FinalContribDf = pd.merge(Feat6Feat7Merge,contribFeat8Df,left_on = 'CONTRIBUTOR',right_on = 'CONTRIBUTOR')


        FinalContribDf.to_csv('yaw_texts.csv',index=False) 

In [4]:
# Arguments: Filename, feature, and required specific features in a list in that order
# Creates: a stacked bar graph
def createStackgraph(*args):
    
    if args[1] == "species/text" or args[1] == "exemplar":
        data=pd.read_csv(args[0],  usecols=(0,1,2,3))
    elif args[1] == "sex/text":
        data=pd.read_csv(args[0],  usecols=(0,1,2,3,4))
    elif args[1] == "quality/text" or args[1] == "age/months":
        data=pd.read_csv(args[0],  usecols=(0,1,2,3,4,5,6))
    elif args[1] == "yaw/text":
        data=pd.read_csv(args[0],  usecols=(0,1,2,3,4,5,6,7,8,9))
    else:
        print("Error")
        
    data = data.sort_values('Sum', ascending=False)
    width = 0.50
    ind = np.arange(56) + 0.75
    
    
    if args[1] == "species/text" or args[1] == "exemplar":
        p1=plt.bar(ind, data.loc[:, args[2]], width, color = 'b') 
        p2=plt.bar(ind, data.loc[:, args[3]], width, color = 'r', bottom=data.loc[:, args[2]])

        plt.legend((p1[0], p2[0]), (args[2], args[3]))
        
    elif args[1] == "sex/text":
        p1=plt.bar(ind, data.loc[:, args[2]], width, color = 'b')
        p2=plt.bar(ind, data.loc[:, args[3]], width, color = 'r', bottom=data.loc[:, args[2]])
        p3=plt.bar(ind, data.loc[:, args[4]], width, color = 'g', bottom=data.loc[:, args[2]] +data.loc[:, args[3]])

        plt.legend((p1[0], p2[0], p3[0]), (args[2], args[3], args[4]))
        
    elif args[1] == "quality/text" or args[1] == "age/months":
        p1=plt.bar(ind, data.loc[:, args[2]], width, color = 'b')
        p2=plt.bar(ind, data.loc[:, args[3]], width, color = 'r', bottom=data.loc[:, args[2]])
        p3=plt.bar(ind, data.loc[:, args[4]], width, color = 'g', bottom=data.loc[:, args[2]] +data.loc[:, args[3]])
        p4=plt.bar(ind, data.loc[:, args[5]], width, color = 'y', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]])
        p5=plt.bar(ind, data.loc[:, args[6]], width, color = 'k', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]] +data.loc[:, args[5]])
        
        plt.legend((p1[0], p2[0], p3[0], p4[0], p5[0]), (args[2], args[3], args[4], args[5], args[6]))
        
    elif args[1] == "yaw/text":
        p1=plt.bar(ind, data.loc[:, args[2]], width, color = 'b')
        p2=plt.bar(ind, data.loc[:, args[3]], width, color = 'r', bottom=data.loc[:, args[2]])
        p3=plt.bar(ind, data.loc[:, args[4]], width, color = 'g', bottom=data.loc[:, args[2]] +data.loc[:, args[3]])
        p4=plt.bar(ind, data.loc[:, args[5]], width, color = 'y', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]])
        p5=plt.bar(ind, data.loc[:, args[6]], width, color = 'k', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]] +data.loc[:, args[5]])
        p6=plt.bar(ind, data.loc[:, args[7]], width, color = 'c', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]] +data.loc[:, args[5]] + data.loc[:, args[6]])
        p7=plt.bar(ind, data.loc[:, args[8]], width, color = 'm', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]] +data.loc[:, args[5]] + data.loc[:, args[6]]+ data.loc[:, args[7]])
        p8=plt.bar(ind, data.loc[:, args[9]], width,color = 'tan', bottom=data.loc[:, args[2]] +data.loc[:, args[3]] +data.loc[:, args[4]] +data.loc[:, args[5]] + data.loc[:, args[6]]+ data.loc[:, args[7]] +data.loc[:, args[8]])

        plt.legend((p1[0], p2[0], p3[0], p4[0], p5[0], p6[0], p7[0], p8[0]), (args[2], args[3], args[4], args[5], args[6], args[7], args[8], args[9]))
    else:
        print("error")
        
    plt.xticks(ind, data.loc[:, 'CONTRIBUTOR'], rotation=90)
    plt.rcParams['xtick.labelsize'] = 3 #line doesn't work anymore, tried all night an nothing, doesn't change the font at all
    plt.ylabel('Total Number of Animals')
    plt.xlabel('Contributor')

    #plt.show() #get rid of to save image
    
    plt.savefig(str("../"+args[2] +"helo_expt2.png"),bbox_inches='tight') # can't put feature because it says can't find directory ex. sex/text

In [5]:
# Arguments : CSV file
# Returns : None
# Creates: csv file with sum column

def sumRows(filename, header=False):  
    with open(filename,'r') as csvfile:
        with open('sum' + filename, 'w') as csvoutput:
            writer = csv.writer(csvoutput, lineterminator='\n')
            reader = csv.reader(csvfile)

            all = []
            row = next(reader)
            row.append('Sum')
            all.append(row)

            for row in reader:
                rowtotal = 0
                for column in row[1:]:
                    rowtotal += int(column)
                row.append(rowtotal)
                all.append(row)

            writer.writerows(all)

In [6]:
# Arguments : csv_file , Required Specific Feature
# Accepted Specific Features: sex_texts = "Male", "Female", "UNKNOWN SEX", etc.
# Creates: plot
def creategraph(csv_file, specific_feature):
    data = pd.read_csv(csv_file, sep=',',header=0, index_col =0) #csv_file

    data.plot(kind='bar')
    plt.rcParams['xtick.labelsize'] = 5
    plt.ylabel('Number of ' + specific_feature + ' taken')
    plt.xlabel('Contributor')
    plt.title('Contributor to '+ specific_feature + ' Totals')

    #plt.show()
    
    specific_feature = 'image_contributor_tag'
    
    plt.savefig(str("../"+specific_feature+"_expt2.png")) #,bbox_inches='tight'